## Importar librerías

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import StratifiedKFold
from sklearn_genetic import GASearchCV
from sklearn_genetic import ExponentialAdapter
from sklearn_genetic.space import Integer, Categorical, Continuous
from sklearn_genetic.callbacks import ProgressBar, ConsecutiveStopping
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from concurrent.futures import ThreadPoolExecutor, wait
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

## Cargar y preparar datos

In [ ]:
# Cargar
tester_files_folder = r"C:\Users\Administrador\AppData\Roaming\MetaQuotes\Terminal\Common\Files"
df_buy = pd.read_csv(os.path.join(tester_files_folder, "buy_training_dataset.csv"))
df_sell = pd.read_csv(os.path.join(tester_files_folder, "sell_training_dataset.csv"))
print(f"Buy -> Trades: {df_buy.shape[0]} Features: {df_buy.shape[1]-1}")
print(f"Sell -> Trades: {df_sell.shape[0]} Features: {df_sell.shape[1]-1}")

In [ ]:
# Aleatorizar muestras (compras)
df_buy = df_buy.sample(frac=1).reset_index(drop=True)
df_buy

In [ ]:
# Aleatorizar muestras (ventas)
df_sell = df_sell.sample(frac=1).reset_index(drop=True)
df_sell

In [5]:
# Chequear y limpiar
if(df_buy.isna().values.any()):
    df_buy=df_buy.dropna()
if(df_sell.isna().values.any()):
    df_sell=df_sell.dropna()

In [6]:
# Preparación de los datos de compra
X_buy_train = df_buy.drop(columns='target').values
y_buy_train = df_buy['target'].values
# Preparación de los datos de venta
X_sell_train = df_sell.drop(columns='target').values
y_sell_train = df_sell['target'].values
# Número de características
n_features = X_buy_train.shape[1]

## Entrenar modelos

In [7]:
def train_model_buy(X_buy_train, y_buy_train, param_grid, cv, crossover_adapter, mutation_adapter):
    try:
        # Definir el pipeline con placeholders (compras)
        pipe_buy = Pipeline([
            ('scaler', RobustScaler()),
            ('dim_reducer', KernelPCA(kernel='rbf')),
            ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
        ])
        # Definir algoritmo genético
        evolutionary_search_buy = GASearchCV(
            estimator=pipe_buy,
            cv=cv,
            scoring='accuracy',
            population_size=100,
            generations=50,
            tournament_size=5,
            elitism=True,
            crossover_probability=crossover_adapter,
            mutation_probability=mutation_adapter,
            param_grid=param_grid,
            criteria='max',
            algorithm='eaMuPlusLambda',
            n_jobs=-1,
            verbose=True,
            keep_top_k=50
        )
        # Entrenar el modelo
        evolutionary_search_buy.fit(X_buy_train, y_buy_train, callbacks=[ProgressBar(), ConsecutiveStopping(generations=1, metric='fitness')])
        # Obtener el mejor estimador
        model_buy = evolutionary_search_buy.best_estimator_
        # Visualizar resultados
        print(evolutionary_search_buy.best_params_)
        print(evolutionary_search_buy.best_score_)
    except Exception as e:
        print(f"Error en train_model_buy: {e}")
        raise
    return model_buy


In [8]:
def train_model_sell(X_sell_train, y_sell_train, param_grid, cv, crossover_adapter, mutation_adapter):
    try:
        # Definir el pipeline con placeholders (ventas)
        pipe_sell = Pipeline([
            ('scaler', RobustScaler()),
            ('dim_reducer', KernelPCA(kernel='rbf')),
            ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
        ])
        # Definir algoritmo genético
        evolutionary_search_sell = GASearchCV(
            estimator=pipe_sell,
            cv=cv,
            scoring='accuracy',
            population_size=100,
            generations=50,
            tournament_size=5,
            elitism=True,
            crossover_probability=crossover_adapter,
            mutation_probability=mutation_adapter,
            param_grid=param_grid,
            criteria='max',
            algorithm='eaMuPlusLambda',
            n_jobs=-1,
            verbose=True,
            keep_top_k=50
        )
        # Entrenar el modelo
        evolutionary_search_sell.fit(X_sell_train, y_sell_train, callbacks=[ProgressBar(), ConsecutiveStopping(generations=1, metric='fitness')])
        # Obtener el mejor estimador
        model_sell = evolutionary_search_sell.best_estimator_
        # Visualizar resultados
        print(evolutionary_search_sell.best_params_)
        print(evolutionary_search_sell.best_score_)
    except Exception as e:
        print(f"Error en train_model_buy: {e}")
        raise
    return model_sell

In [9]:
# Preparación de los datos de compra
X_buy_train = df_buy.drop(columns='target').values
y_buy_train = df_buy['target'].values
# Preparación de los datos de venta
X_sell_train = df_sell.drop(columns='target').values
y_sell_train = df_sell['target'].values
# Número de características
n_features = X_buy_train.shape[1]

In [10]:
# Definir parámetros genéticos
crossover_adapter = ExponentialAdapter(initial_value=0.1, end_value=0.9, adaptive_rate=0.1)
mutation_adapter = ExponentialAdapter(initial_value=0.9, end_value=0.1, adaptive_rate=0.1)
cv = StratifiedKFold(n_splits=5, shuffle=True)
# Definir espacio de hiperparámetros
param_grid = {
    'dim_reducer__n_components': Integer(1, n_features),
    'dim_reducer__gamma': Continuous(1e-4, 1e-1, distribution='log-uniform'),
    'xgb__n_estimators': Integer(50, 500),
    'xgb__max_depth': Integer(3, 10),
    'xgb__learning_rate': Continuous(0.01, 0.3),
    'xgb__subsample': Continuous(0.6, 1.0),
    'xgb__colsample_bytree': Continuous(0.6, 1.0),
    'xgb__gamma': Continuous(0, 0.5),
    'xgb__min_child_weight': Integer(1, 10),
    'xgb__reg_alpha': Continuous(0, 1.0),
    'xgb__reg_lambda': Continuous(0, 1.0)
}

In [ ]:

# Entrenar modelos simultáneamente
with ThreadPoolExecutor(max_workers=2) as executor:
    # enviar tareas de entrenamiento
    future_buy = executor.submit(train_model_buy, X_buy_train, y_buy_train, param_grid, cv, crossover_adapter, mutation_adapter)
    future_sell = executor.submit(train_model_sell, X_sell_train, y_sell_train, param_grid, cv, crossover_adapter, mutation_adapter)
    # esperar a que todas las tareas terminen
    futures = [future_buy, future_sell]
    print("Esperando que las tareas finalicen...")
    wait(futures)
    print("¡Todas las tareas han terminado!")
    # Obtener resultados una vez que ambas tareas han terminado
    model_buy = future_buy.result()
    model_sell = future_sell.result()

## Exportar modelos a formato ONNX

In [ ]:
update_registered_converter(
    XGBClassifier,
    "XGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={'nocl': [False], 'zipmap': [True, False, 'columns']}
)
model_buy_onnx = convert_sklearn(
    model_buy,
    'pipeline_buy_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)
model_sell_onnx = convert_sklearn(
    model_sell,
    'pipeline_sell_xgboost',
    [('input', FloatTensorType([None, X_buy_train.shape[1]]))],
    target_opset={'': 12, 'ai.onnx.ml': 2}
)

In [21]:
mql5_files_folder = "C:/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files"
with open(os.path.join(mql5_files_folder, "model_buy.onnx"), 'wb') as f:
    f.write(model_buy_onnx.SerializeToString())
with open(os.path.join(mql5_files_folder, "model_sell.onnx"), 'wb') as f:
    f.write(model_sell_onnx.SerializeToString())